In [9]:
def decimal_to_posit6_0(x: float) -> str:
    """
    Convert decimal (float) to posit(6,0) bitstring.
    Works for positive and negative numbers.
    """

    nbits = 6
    es = 0
    useed = 2 ** (2 ** es)  # useed = 2

    # Special cases
    if x == 0:
        return "0" * nbits  # zero
    if x != x:  # NaN
        return "1" + "0" * (nbits - 1)  # NaR (Not a Real)

    # Remember sign
    sign = 0 if x > 0 else 1
    x = abs(x)

    # regime calculation
    k = 0
    value = 1.0
    while x >= useed:
        x /= useed
        k += 1
    while x < 1.0:
        x *= useed
        k -= 1

    # regime bits
    if k >= 0:
        regime = "1" * (k + 1) + "0"
    else:
        regime = "0" * (-k) + "1"

    # keep within available bits
    regime = regime[:nbits - 1]

    # fraction bits (remaining)
    max_frac_bits = nbits - 1 - len(regime)
    frac_val = x - 1
    frac_bits = ""
    for i in range(max_frac_bits):
        frac_val *= 2
        if frac_val >= 1:
            frac_bits += "1"
            frac_val -= 1
        else:
            frac_bits += "0"

    # Construct posit without sign
    posit_mag = (regime + frac_bits)[:nbits - 1].ljust(nbits - 1, "0")
    posit_bits = "0" + posit_mag  # temporary sign=0

    # If negative → take two's complement
    if sign == 1:
        val = int(posit_bits, 2)
        val = ((~val) + 1) & ((1 << nbits) - 1)  # two’s complement
        posit_bits = format(val, f"0{nbits}b")

    return "0b"+posit_bits


In [11]:
decimal_to_posit6_0(-20)

'0b100001'